In [1]:
from math import factorial

import os,io
from sympy import Poly,terms_gcd,gcd_list
import re
import random
from sympy import factorint,primerange
from fractions import Fraction

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

def count_appearances(key,slot):
    # the letter in slot is the Nth appearance, counting from left and starting the count at 1.
    # returns N.
    letter=key[slot]
    return [i for i in find_all(key,letter)].index(slot)+1

def encode(value):
    if value != 0:
        prefix = []
        w = abs(value)
        while w > 0:
            prefix.append(str(w % 1000))
            w = w // 1000
        prefix = prefix[::-1]
    else:
        prefix =['0']
    prefix = (['+'] if value >= 0 else ['-']) + prefix
    return prefix

class fastRandomDict(object):
    # from: https://stackoverflow.com/questions/24630804/how-to-get-a-random-value-in-a-very-large-python-dictionary
    def __init__(self, init_dict):
        self.mydict=init_dict.copy()
        # Create the bidirectional maps from the dictionary, D
        self.keys = self.mydict.keys()
        self.ints = range(len(self.keys))
        self.int_to_key = dict(zip(self.ints, self.keys))
        self.key_to_int = dict(zip(self.keys, self.ints))
    def __getitem__(self, item):
        if item in self.mydict:
            return self.mydict[item]
        else: return None
    def __contains__(self, item):
        return item in self.mydict
    def __len__(self):
        len(self.mydict)
    def add(self, key, value):  # O(1)
        # Add key-value pair (no extra work needed for simply changing the value)
        new_int = len(self.mydict)
        self.mydict[key] = value
        self.int_to_key[new_int] = key
        self.key_to_int[key] = new_int
    def remove(self, key):  # O(1)
        # Update the bidirectional maps then remove the key-value pair
        # Get the two ints and keys.
        if key not in self.key_to_int: return
        key_int = self.key_to_int[key]
        swap_int = len(self.mydict) - 1  # Should be the highest int
        swap_key = self.int_to_key[swap_int]

        # Update the bidirectional maps so that key now has the highest int
        self.key_to_int[key], self.key_to_int[swap_key] = swap_int, key_int
        self.int_to_key[key_int], self.int_to_key[swap_int] = swap_key, key

        # Remove elements from dictionaries
        del self.mydict[key]
        del self.key_to_int[key]
        del self.int_to_key[swap_int]
    def random_key(self):  # O(1)
        # Select a random key from the dictionary using the int_to_key map
        return self.int_to_key[int(len(self.mydict) * random.random())]
    def remove_random(self):  # O(1)
        # Randomly remove a key from the dictionary via the bidirectional maps
        key = self.random_key()
        self.remove(key)
    def pop_random(self):  # O(1)
        try:
        # Randomly pop a key from the dictionary via the bidirectional maps
            key = self.random_key()
            val = self.mydict[key]
            self.remove(key)
        except IndexError:
            print("Error, symbol is exhausted!")
        return key,val
    def pop_random_gen(self, num_to_pop):  # O(1)
        for i in range(num_to_pop):
            yield self.pop_random()
    def pop_inst_gen(self,subdict_size, num_to_gen):
        # Randomly pop instances from the symb
        for i in range(num_to_gen):
            yield {k:v for k, v in self.pop_random_gen(subdict_size)}

def decode(lst):
    if len(lst) < 1:
        return None
    if len(lst) < 1 or (lst[0] != '+' and lst[0] != '-'):
        return None
    if len(lst) == 1:
        return 0
    res = 0
    for x in lst[1:]:
        if not (x.isdigit()):
            return None
        res = res * 1000 + int(x)
    return -res if lst[0] == '-' else res

def readESymb(loop, name):
    assert os.path.isfile(name)
    res = ''
    prefix = 'Esymb'
    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(loop) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res

def cl(text):
    return re.sub(r'\s+', '', text)
    
def convert(loop, filename):
    dev = re.split(":=|SB\(|\)", re.sub('[,*]', '', readESymb(loop, filename)))[1:-1]
    keys = dev[1::2]
    values = [int(re.sub('[+-]$', t[0] + '1', t)) for t in dev[0::2]]
    out_dict = {}
    for k, v in zip(keys, values):
        out_dict[k] = v
    return out_dict

def export_hash(outfile,data):
    file_handler = io.open(outfile, mode="wt", encoding="utf-8")
    for i, (k, v) in enumerate(data.items()):
        prefix1_str = " ".join(k)
        prefix2 = encode(v)
        prefix2_str = " ".join(prefix2)
        file_handler.write(f"{i + 1}|{prefix1_str}\t{prefix2_str}\n")
        file_handler.flush()
    file_handler.close()
    return
    

def read_encoded(path,train=False):
    with io.open(path, mode="r", encoding="utf-8") as f:
        # either reload the entire file, or the first N lines
        # (for the training set)
        if not train:
            lines = [line.rstrip().split("|") for line in f]
        else:
            lines = []
            for i, line in enumerate(f):
                lines.append(line.rstrip().split("|"))
    data = [xy.split("\t") for _, xy in lines]
    data_dict = {''.join(xy[0].split()):decode(xy[1].split()) for xy in data if len(xy) == 2}
    return data_dict

symb={}
symb[6]=convert(6,'../data/EZ6_symb_new_norm')
symb[5]=convert(5,'../data/EZ_symb_new_norm')
symb[4]=convert(4,'../data/EZ_symb_new_norm')
symb[3]=convert(3,'../data/EZ_symb_new_norm')
symb[2]=convert(2,'../data/EZ_symb_new_norm')
symb[1]=convert(1,'../data/EZ_symb_new_norm')

In [6]:

from itertools import permutations, islice
alphabet = ['a', 'b', 'c', 'd', 'e', 'f']
dihedral_table = [list(permutations(alphabet[:3]))[i] + list(permutations(alphabet[3:]))[i] for i in
                  range(len(alphabet))]
cycle_table = [dihedral_table[i] for i in [0, 3, 4]]
flip_table = [dihedral_table[i] for i in [0, 1, 2, 5]]

##########################################################################################
#Operator Library. One-to-one operators:
###########################################################################################

def strike(key, slotset):
    # k: all slots in the operation must be within 'k' of each other
    # assume slotset is sorted, all slots are < len(key),
    # and slotsets are all within k. This is done at slot generation time.
    # can't strike slots that are not in the key!
    slotset=set(slotset)
    strike_key=''.join([k for i,k in enumerate(key) if i not in slotset])
    return strike_key

def swap(key, slotset):
    # swap letters at the given slot.
    if len(slotset) > 2: return
    swap_key=key[:slotset[0]] + key[slotset[1]] + key[(slotset[0]+1):slotset[1]] + key[slotset[0]]
    if slotset[1] != len(key): swap_key += key[(slotset[1]+1):]
    return swap_key

def unstrike(key, slotset, letterset,runlenset=None,targetloop=None):
    # insert runs of the specified letter into the key.
    # careful with lookups here!
    newkey=key
    if not runlenset:
        runlenset=[1]*len(slotset)
    for slot,let,runlen in zip(slotset,letterset,runlenset):
        newkey = newkey[:slot] + let*runlen + newkey[slot:]

    #check to make sure adding runs gets us to the correct loop
    if targetloop and(len(newkey)!=2*len(targetloop)): raise ValueError
    return newkey

def mutate(key, slotset, letterset):
    # Change the letter in the given slot to the given letter.
    keyslots=dict(zip(slotset,letterset))
    return ''.join([elem if i not in slotset else keyslots[i] for i,elem in enumerate(key)])

def subword(key,slotset):
    # get the subword that spans the two slots described,
    # INCLUDING the last letter
    if len(slotset) > 2: return
    return key[slotset[0]:slotset[1]]+key[slotset[1]]

def slot_select(key,slotset):
    return ''.join([key[i] for i in slotset])

def dihedral(key,rot_ind):
    # rot_ind is 1,2,3,4 or 5
    if rot_ind not in {1,2,3,4,5}: print(rot_ind); raise ValueError
    word_idx = [alphabet.index(l) for l in [*key]]
    image = ''.join([dihedral_table[rot_ind][idx] for idx in word_idx])
    return image

def cycle(key,rot_ind):
    #rot_ind is 3 or 4
    if rot_ind not in {3,4}: print(rot_ind); raise ValueError
    word_idx = [alphabet.index(l) for l in [*key]]
    cycle = ''.join([dihedral_table[rot_ind][idx] for idx in word_idx])
    return cycle

def flip(key,rot_ind):
    #rot_ind is 1 or 2, 5
    if rot_ind not in {1,2,5}: print(rot_ind); raise ValueError
    word_idx = [alphabet.index(l) for l in [*key]]
    flip = ''.join([flip_table[rot_ind][idx] for idx in word_idx])
    return flip

def partial_dihedral(key, slotset, rot_ind):
    if len(slotset) != 2: raise ValueError
    return key[:slotset[0]]+dihedral(key[slotset[0]:slotset[1]],rot_ind)+key[slotset[1]:]

def partial_cycle(key, slotset, rot_ind):
    if len(slotset) != 2: raise ValueError
    return key[:slotset[0]]+cycle(key[slotset[0]:slotset[1]],rot_ind)+key[slotset[1]:]

def partial_flip(key, slotset, rot_ind):
    if len(slotset) != 2: raise ValueError
    return key[:slotset[0]]+flip(key[slotset[0]:slotset[1]],rot_ind)+key[slotset[1]:]

###########################################################################
# one-to-many operators (allstrikes,etc.)
##########################################################################

def all_partial_dihedral_operator(key,slotset):
    prefix=key[:slotset[0]]
    substr=key[slotset[0]:slotset[1]]
    suffix=key[slotset[1]:]
    return [prefix+im+suffix for im in all_dihedral_operator(substr)]

def all_partial_cycle_operator(key,slotset):
    prefix=key[:slotset[0]]
    substr=key[slotset[0]:slotset[1]]
    suffix=key[slotset[1]:]
    return [prefix+im+suffix for im in all_cycle_operator(substr)]

def all_partial_flip_operator(key,slotset):
    prefix=key[:slotset[0]]
    substr=key[slotset[0]:slotset[1]]
    suffix=key[slotset[1]:]
    return [prefix+im+suffix for im in all_flip_operator(substr)]

def all_dihedral_operator(word):
    word_idx = [alphabet.index(l) for l in [*word]]
    dihedral_images = [''.join([dihedral_table[row][idx] for idx in word_idx]) for row in range(1,len(alphabet))]
    return dihedral_images

def all_cycle_operator(word):
    word_idx = [alphabet.index(l) for l in [*word]]
    cycle_images = [''.join([cycle_table[row][idx] for idx in word_idx]) for row in range(1,int(len(alphabet) / 2))]
    return cycle_images

def all_flip_operator(word):
    my_images = all_dihedral_operator(word)
    word_idx = [alphabet.index(l) for l in [*word]]
    cycle_images = [''.join([cycle_table[row][idx] for idx in word_idx]) for row in range(1,int(len(alphabet) / 2))]
    return [im for im in my_images if im not in cycle_images]

#def all_strike_operator(word):


##########################################################################
# slot selectors. Not used in instance gen, but can be in future- need to figure out
# more about how python handles passing compositions of functions
##########################################################################

def get_rundict(sent):
    thischar=0
    runs={}
    counter=1
    for i in range(len(sent)):
        if sent[i]==thischar:
            counter += 1
        else:
            thischar=sent[i]
            if i != 0:
                if not sent[i-1] in runs: runs[sent[i-1]]=[]
                runs[sent[i-1]]+=[(i-1,f'r{counter}')]
            counter = 1
    
    if not sent[i-1] in runs: runs[sent[i-1]]=[]
    runs[sent[i-1]]+=[(i-1,f'r{counter}')]
    return runs

def get_first_nth_appearance(key, substr, n):
    if n < 1: raise ValueError
    #get the nth occurrence of a substring (assumes it appears >= n times)
    allapps=[i for i in find_all(key, substr)]
    if n > len(allapps): return None
    first=allapps[n-1]
    return first

def get_last_nth_appearance(key, substr, n):
    if n < 1: raise ValueError
    #get the nth occurrence from the right of a substring (assumes it appears >= n times)
    allapps=[i for i in find_all(key, substr)]
    if n > len(allapps): return None
    last=allapps[-n]
    return last

def get_samerun_slots(sent,seedslot):
    thischar=sent[seedslot]
    counter=1
    good=[]
    #get all slots that are in the same "run" as the given slot
    r1,r2=0,0
    #forward
    for i in range(len(sent)-seedslot+1):
        if sent[seedslot+i]==thischar: continue  
        else:
            r1 = seedslot+i-1
            break
    #back
    for i in range(seedslot):
        if sent[seedslot-i]==thischar: continue  
        else:
            r2 = seedslot-i-1
            break
    return [r1+i for i in range(0,r2-r1-1)]

def get_runbound_slots(sent,seedslot):
    #get the slots that bookend the given "run".
    run=get_samerun_slots(sent,seedslot)
    return [run[0]-1,run[-1]+1]

def get_first_nth_run_of(k,let,n):
    if n < 1: return
    else: return get_rundict(k)[let][n-1][0]

def get_last_nth_run_of(k,let,n):
    if n < 1: return
    return get_rundict(k)[let][::-1][n-1][0]
    
    
    

In [5]:
get_first_nth_appearance("abbccbb","b",1)

1

In [ ]:
#strike out letters in positions "pos1", "pos2" from all keys in the provided symb.

def strike_pos(words2strike,symb_low,pos1,pos2,return_meta=False):
    #return one instance of Symb class to do ops with and one that has metadata
    out_coeffs_only={}
    out_coeffs_metadata={}
    if not (pos1 < pos2): raise ValueError 
    for key in symb:
        if pos1 > len(key) or pos2 > len(key): raise ValueError
        strike_key=key[:pos1] + key[pos1+1:pos2]+ key[pos2+1:]
        

def make_strikeseq(seed,i,j):
    k=seed
    L=len(k)/2
    seq=[]
    while L > 0:
        k_new=strike_pos(k,symb[L],i,j)
        coef=symb[L][k]
        k=k_new
        L=L-1
    

In [91]:
import numpy as np
from scipy.interpolate import lagrange
import math

deg is 2 less than num checks

def is_basicray(seq,startloop):
    for k in range(startloop+1):
        #scale seq:
        oseq,lseq=[],[]
        for i,elem in enumerate(seq):
            L=i+startloop
            oseq+=[Fraction(elem*factorial(L-k),pow(-2,L)*factorial(2*L-2*k))]
            lseq += [L]
        poly = lagrange(lseq, oseq)
        outseq=[round(number) if math.isclose(number, round(number)) else 0 for number in poly.coef][::-1]
        for i,elem in enumerate(seq):
            L=i+startloop
            if sum([jelem*pow(L,j) for j,jelem in enumerate(outseq) if jelem is not None]) == elem:
                p='+'.join([f'{c}*L^{j}' if j !=0 else f'{c}' for j,c in enumerate(outseq) if c !=0])
                return f'[(-2)^(L-{k})]*[(2L-{2*k}))!/(L-{k})!]*[{p}]'
    return False

def is_seqray():

In [ ]:
def make_strikeray(seq,startloop):
    
    abs_slots:
        
    get_first_nth_run_of:
        
    get_last_nth_run_of:
        
    +i,-i
    
    get_runbound_slots:
        
    
    
    return seq

In [92]:
is_basicray(l,1)

'[(-2)^(L-1)]*[(2L-2))!/(L-1)!]*[-2]'